## Use taxonomic read classifications from MEGAN6 to extract day and treatment taxa-specific NanoPore FastQ reads

In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Tue 13 Oct 2020 09:49:02 AM PDT
------------

Distributor ID:	Ubuntu
Description:	Ubuntu 20.04.1 LTS
Release:	20.04
Codename:	focal

------------
HOSTNAME: 
mephisto

------------
Computer Specs:

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   36 bits physical, 48 bits virtual
CPU(s):                          4
On-line CPU(s) list:             0-3
Thread(s) per core:              2
Core(s) per socket:              2
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           58
Model name:                      Intel(R) Core(TM) i7-3517U CPU @ 1.90GHz
Stepping:                        9
CPU MHz:                         2917.625
CPU max MHz:                     3000.0000
CPU min MHz:                     800.0000
BogoMIPS:             

No LSB modules are available.


### Set variables

In [4]:
# Set data directories
%env muscle_fasta_dir=/home/samb/analyses/20201007_cbai_megan-read-extractions_201002558-2729-Q7
%env hemo_fasta_dir=/home/samb/analyses/20201007_cbai_megan-read-extractions_6129-403-26-Q7
%env fastq_dir=/home/samb/data/C_bairdi/DNAseq
%env wd=/home/samb/analyses

# Programs
%env seqtk=/home/samb/programs/seqtk_1.3-r115/seqtk

# File naming
%env suffix=megan.fq


env: muscle_fasta_dir=/home/samb/analyses/20201007_cbai_megan-read-extractions_201002558-2729-Q7
env: hemo_fasta_dir=/home/samb/analyses/20201007_cbai_megan-read-extractions_6129-403-26-Q7
env: fastq_dir=/home/samb/data/C_bairdi/DNAseq
env: wd=/home/samb/analyses
env: seqtk=/home/samb/programs/seqtk_1.3-r115/seqtk
env: suffix=megan.fq


#### Input data are here:

FastAs: 

- https://gannet.fish.washington.edu/Atumefaciens/20201007_cbai_megan-read-extractions_201002558-2729-Q7/

- https://gannet.fish.washington.edu/Atumefaciens/20201007_cbai_megan-read-extractions_6129-403-26-Q7/

FastQs:

- https://gannet.fish.washington.edu/Atumefaciens/20200928_cbai_nanofilt_Q7_20102558-2729_nanopore-data/20200928_cbai_nanopore_20102558-2729_quality-7.fastq

- https://gannet.fish.washington.edu/Atumefaciens/20200928_cbai_nanofilt_Q7_6129_403_26_nanopore-data/20200928_cbai_nanopore_6129_403_26_quality-7.fastq

### Extract taxa-specific reads from FastQ files

Use FastA IDs from MEGAN6 taxonomic read extraction FastAs to pull out appropriate reads from each taxa.

In [8]:
%%bash

timestamp=$(date +%Y%m%d)


for directory in ${muscle_fasta_dir} ${hemo_fasta_dir}
do
	# Get sample name
	sample=$(echo "${directory}" | cut -d "_" -f 4)
    
    # Make new directory and change to that directory ("$_" means use previous command's argument)
    mkdir --parents "${wd}"/"${timestamp}"_"${sample}"_megan-reads \
    && cd "$_" || exit


	######################################################
	# Create FastA IDs list to use for sequence extraction
	######################################################
	for fai in "${directory}"/*.fai
	do
      # Get species
      if [[ "${sample}" = "201002558-2729-Q7" ]]; then
        species=$(echo "${fai##*/}" | awk -F [.-] '{print $5}')
      else
        species=$(echo "${fai##*/}" | awk -F [.-] '{print $6}')
      fi
    
      # Set output FastQ filenames
      prefix=${timestamp}_${sample}_${species}

	  # Set seqtk list filename
	  seqtk_list=${prefix}_seqtk-read-id-list
        
      echo "Pulling FastA IDs from ${fai}"
      echo ""
        
      # Parse FastA IDs from FastA index file
      awk '{print $1}' "${fai}" | sort -u >> "${seqtk_list}"
    
           
      echo "Extracting reads from ${fastq}."
      echo ""
        
      out="${prefix}_${suffix}"
    
      for fastq in ${fastq_dir}/*.fastq
      do
        # Extract corresponding reads using seqtk FastA ID list
  	    ${seqtk} subseq "${fastq}" "${seqtk_list}" >> "${out}"
      done
 
      echo "Writing reads to ${out}"
      echo ""
      echo ""
    
	done

    
    echo ""
    echo "Done with read extractions"
    echo ""
    echo "-------------------------------------"
    echo ""

    # Print working directory and list files
    pwd
	ls -ltrh
    echo ""
    echo "-------------------------------------"
    echo ""
done


Pulling FastA IDs from /home/samb/analyses/20201007_cbai_megan-read-extractions_201002558-2729-Q7/201002558-2729-Q7_summarized-reads-Aquifex_sp..fasta.fai

Extracting reads from .

Writing reads to 20201013_201002558-2729-Q7_Aquifex_sp_megan.fq


Pulling FastA IDs from /home/samb/analyses/20201007_cbai_megan-read-extractions_201002558-2729-Q7/201002558-2729-Q7_summarized-reads-Arthropoda.fasta.fai

Extracting reads from /home/samb/data/C_bairdi/DNAseq/20200928_cbai_nanopore_6129_403_26_quality-7.fastq.

Writing reads to 20201013_201002558-2729-Q7_Arthropoda_megan.fq


Pulling FastA IDs from /home/samb/analyses/20201007_cbai_megan-read-extractions_201002558-2729-Q7/201002558-2729-Q7_summarized-reads-Enterospora_canceri.fasta.fai

Extracting reads from /home/samb/data/C_bairdi/DNAseq/20200928_cbai_nanopore_6129_403_26_quality-7.fastq.

Writing reads to 20201013_201002558-2729-Q7_Enterospora_canceri_megan.fq


Pulling FastA IDs from /home/samb/analyses/20201007_cbai_megan-read-extractions